In [129]:
import numpy as np
import matplotlib.pyplot as plt
import re
import seaborn as sb
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [130]:
import pandas as pd
import itertools
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, HashingVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import PassiveAggressiveClassifier, SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

In [131]:
import sqlite3
import pandas as pd

conn = sqlite3.connect("Fake_News_DB.db")

In [132]:
data = pd.read_sql("SELECT * from fake_news", conn)
data

,id,url,title,content,author,date,tags,type_of_article
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Россиянам запретят выкладывать в интернет непр...,В Госдуму внесён законопроект о «Защите положи...,Виталий Манн,2019-12-19,Политика,fake
1,1,https://panorama.pub/28365-amerikanskij-genera...,Американский генерал заявил о готовности стрел...,Командующий войсками Округа Колумбия генерал-л...,Бронислав Рубенштейн,2019-12-19,Политика,fake
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Дональд Трамп похвалил Сталина,"Президент США Дональд Трамп, отвечая на вопрос...",Виталий Манн,2019-12-19,Политика,fake
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Формальным основанием для импичмента Трампа ст...,Палата представителей США определилась с форму...,Бронислав Рубенштейн,2019-12-18,Политика,fake
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Иранские моряки по ошибке подавили протесты в ...,"Признавая, что они были полностью сбиты массой...",Диана Арутюнян,2019-12-18,Политика,fake
5,5,https://panorama.pub/28198-prezident-turkmenis...,Президент Туркменистана победил на национально...,Президент Туркменистана Гурбангулы Бердымухаме...,Борис Гонтермахер,2019-12-16,Политика,fake
6,6,https://panorama.pub/28130-rosimushhestvo-pred...,Росимущество предоставит Горбачёву дачу в Форосе,Росимущество оперативно и положительно отреаги...,Виталий Манн,2019-12-16,Политика,fake
7,7,https://panorama.pub/28100-gorbachyov-poprosil...,Михаил Горбачёв попросил выделить ему новую го...,Первый и единственный президент СССР Михаил Го...,Бронислав Рубенштейн,2019-12-15,Политика,fake
8,8,https://panorama.pub/27843-vdol-granitsy-rossi...,Вдоль границ России создадут 500-километровую ...,Министерство обороны представило проект иннова...,Марк Хлестаков,2019-12-14,Политика,fake
9,9,https://panorama.pub/27987-rossiya-zakonchila-...,Россия закончила выплаты США за поставки оружи...,В начале декабря Центробанк перевёл на счёт Го...,Антон Мицкелюнас,2019-12-13,Политика,fake


In [133]:
# merge title and content for further text preprocessing

data['newstext'] = data[ ['title' , 'content'] ].apply( lambda t : ' '.join( t ) , axis = 1)

In [134]:
data.newstext.head(15)

0     Россиянам запретят выкладывать в интернет непр...
1     Американский генерал заявил о готовности стрел...
2     Дональд Трамп похвалил Сталина Президент США Д...
3     Формальным основанием для импичмента Трампа ст...
4     Иранские моряки по ошибке подавили протесты в ...
5     Президент Туркменистана победил на национально...
6     Росимущество предоставит Горбачёву дачу в Форо...
7     Михаил Горбачёв попросил выделить ему новую го...
8     Вдоль границ России создадут 500-километровую ...
9     Россия закончила выплаты США за поставки оружи...
10    Борис Джонсон: «Будем проводить референдумы по...
11    Рассекречено дело НКВД 1938 года на путешестве...
12    Штаты, не легализовавшие однополые браки, не б...
13    Горбачёв поддержал отстранение России от Олимп...
14    Минспорт запретил чиновникам WADA появляться н...
Name: newstext, dtype: object

In [135]:
# drop content and title
data.drop(columns= "title", axis=1, inplace=True)
data.drop(columns="content", axis=1, inplace=True)

In [136]:
data['newstext'] = data['newstext'].apply( lambda t : " ".join( t.split() ) )
data.head(15)


,id,url,author,date,tags,type_of_article,newstext
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Виталий Манн,2019-12-19,Политика,fake,Россиянам запретят выкладывать в интернет непр...
1,1,https://panorama.pub/28365-amerikanskij-genera...,Бронислав Рубенштейн,2019-12-19,Политика,fake,Американский генерал заявил о готовности стрел...
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Виталий Манн,2019-12-19,Политика,fake,Дональд Трамп похвалил Сталина Президент США Д...
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Бронислав Рубенштейн,2019-12-18,Политика,fake,Формальным основанием для импичмента Трампа ст...
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Диана Арутюнян,2019-12-18,Политика,fake,Иранские моряки по ошибке подавили протесты в ...
5,5,https://panorama.pub/28198-prezident-turkmenis...,Борис Гонтермахер,2019-12-16,Политика,fake,Президент Туркменистана победил на национально...
6,6,https://panorama.pub/28130-rosimushhestvo-pred...,Виталий Манн,2019-12-16,Политика,fake,Росимущество предоставит Горбачёву дачу в Форо...
7,7,https://panorama.pub/28100-gorbachyov-poprosil...,Бронислав Рубенштейн,2019-12-15,Политика,fake,Михаил Горбачёв попросил выделить ему новую го...
8,8,https://panorama.pub/27843-vdol-granitsy-rossi...,Марк Хлестаков,2019-12-14,Политика,fake,Вдоль границ России создадут 500-километровую ...
9,9,https://panorama.pub/27987-rossiya-zakonchila-...,Антон Мицкелюнас,2019-12-13,Политика,fake,Россия закончила выплаты США за поставки оружи...


In [137]:
# feature extraction: number of UPPERCASE Letters

def uppercase_count(t):
    upper_list = []
    for word in t.split():
        if word.isupper():
            upper_list.append(word)
    return len(upper_list)


data['uppercase_count'] = data.newstext.apply(lambda t : uppercase_count(t))
data.head(15)

,id,url,author,date,tags,type_of_article,newstext,uppercase_count
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Виталий Манн,2019-12-19,Политика,fake,Россиянам запретят выкладывать в интернет непр...,7
1,1,https://panorama.pub/28365-amerikanskij-genera...,Бронислав Рубенштейн,2019-12-19,Политика,fake,Американский генерал заявил о готовности стрел...,5
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Виталий Манн,2019-12-19,Политика,fake,Дональд Трамп похвалил Сталина Президент США Д...,5
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Бронислав Рубенштейн,2019-12-18,Политика,fake,Формальным основанием для импичмента Трампа ст...,1
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Диана Арутюнян,2019-12-18,Политика,fake,Иранские моряки по ошибке подавили протесты в ...,4
5,5,https://panorama.pub/28198-prezident-turkmenis...,Борис Гонтермахер,2019-12-16,Политика,fake,Президент Туркменистана победил на национально...,2
6,6,https://panorama.pub/28130-rosimushhestvo-pred...,Виталий Манн,2019-12-16,Политика,fake,Росимущество предоставит Горбачёву дачу в Форо...,3
7,7,https://panorama.pub/28100-gorbachyov-poprosil...,Бронислав Рубенштейн,2019-12-15,Политика,fake,Михаил Горбачёв попросил выделить ему новую го...,2
8,8,https://panorama.pub/27843-vdol-granitsy-rossi...,Марк Хлестаков,2019-12-14,Политика,fake,Вдоль границ России создадут 500-километровую ...,3
9,9,https://panorama.pub/27987-rossiya-zakonchila-...,Антон Мицкелюнас,2019-12-13,Политика,fake,Россия закончила выплаты США за поставки оружи...,4


In [138]:
# Feature extraction : number of words from
def word_count(t):
    words = t.split(' ')
    return (len(words))

data['word_count'] = data.newstext.apply(lambda t : word_count(t))
data.head(15)

,id,url,author,date,tags,type_of_article,newstext,uppercase_count,word_count
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Виталий Манн,2019-12-19,Политика,fake,Россиянам запретят выкладывать в интернет непр...,7,215
1,1,https://panorama.pub/28365-amerikanskij-genera...,Бронислав Рубенштейн,2019-12-19,Политика,fake,Американский генерал заявил о готовности стрел...,5,181
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Виталий Манн,2019-12-19,Политика,fake,Дональд Трамп похвалил Сталина Президент США Д...,5,163
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Бронислав Рубенштейн,2019-12-18,Политика,fake,Формальным основанием для импичмента Трампа ст...,1,183
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Диана Арутюнян,2019-12-18,Политика,fake,Иранские моряки по ошибке подавили протесты в ...,4,231
5,5,https://panorama.pub/28198-prezident-turkmenis...,Борис Гонтермахер,2019-12-16,Политика,fake,Президент Туркменистана победил на национально...,2,143
6,6,https://panorama.pub/28130-rosimushhestvo-pred...,Виталий Манн,2019-12-16,Политика,fake,Росимущество предоставит Горбачёву дачу в Форо...,3,147
7,7,https://panorama.pub/28100-gorbachyov-poprosil...,Бронислав Рубенштейн,2019-12-15,Политика,fake,Михаил Горбачёв попросил выделить ему новую го...,2,143
8,8,https://panorama.pub/27843-vdol-granitsy-rossi...,Марк Хлестаков,2019-12-14,Политика,fake,Вдоль границ России создадут 500-километровую ...,3,209
9,9,https://panorama.pub/27987-rossiya-zakonchila-...,Антон Мицкелюнас,2019-12-13,Политика,fake,Россия закончила выплаты США за поставки оружи...,4,182


In [139]:
#  Feature : Average word length for content
def avg_word_length(t):
    words = t.split()
    return ( sum( len(word) for word in words ) / len(words))

data['avg_word_len'] = data.newstext.apply(lambda t : avg_word_length(t))
data.head(15)

,id,url,author,date,tags,type_of_article,newstext,uppercase_count,word_count,avg_word_len
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Виталий Манн,2019-12-19,Политика,fake,Россиянам запретят выкладывать в интернет непр...,7,215,7.060465
1,1,https://panorama.pub/28365-amerikanskij-genera...,Бронислав Рубенштейн,2019-12-19,Политика,fake,Американский генерал заявил о готовности стрел...,5,181,5.911602
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Виталий Манн,2019-12-19,Политика,fake,Дональд Трамп похвалил Сталина Президент США Д...,5,163,5.889571
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Бронислав Рубенштейн,2019-12-18,Политика,fake,Формальным основанием для импичмента Трампа ст...,1,183,6.573770
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Диана Арутюнян,2019-12-18,Политика,fake,Иранские моряки по ошибке подавили протесты в ...,4,231,6.346320
5,5,https://panorama.pub/28198-prezident-turkmenis...,Борис Гонтермахер,2019-12-16,Политика,fake,Президент Туркменистана победил на национально...,2,143,7.314685
6,6,https://panorama.pub/28130-rosimushhestvo-pred...,Виталий Манн,2019-12-16,Политика,fake,Росимущество предоставит Горбачёву дачу в Форо...,3,147,6.789116
7,7,https://panorama.pub/28100-gorbachyov-poprosil...,Бронислав Рубенштейн,2019-12-15,Политика,fake,Михаил Горбачёв попросил выделить ему новую го...,2,143,6.083916
8,8,https://panorama.pub/27843-vdol-granitsy-rossi...,Марк Хлестаков,2019-12-14,Политика,fake,Вдоль границ России создадут 500-километровую ...,3,209,7.167464
9,9,https://panorama.pub/27987-rossiya-zakonchila-...,Антон Мицкелюнас,2019-12-13,Политика,fake,Россия закончила выплаты США за поставки оружи...,4,182,6.175824


In [140]:
# Remove frequent words
all_words = ' '.join(data.newstext).split()

# let's keep the threshold of 28 K which almost equal to number of data instances in the dataset
freq_words = pd.Series(all_words).value_counts()[:16]

In [141]:
freq_words

в      35715
и      23471
на     15775
что    10324
с       9858
не      8197
—       6282
по      5559
из      5448
о       5315
-       4859
В       4699
для     4368
за      3438
к       3338
а       3162
dtype: int64

In [142]:
# remove freq_words
data.newstext = data.newstext.apply(lambda t: " ".join(word for word in t.split() if word not in freq_words)) 
data.newstext.head(5)

0    Россиянам запретят выкладывать интернет неприг...
1    Американский генерал заявил готовности стрелят...
2    Дональд Трамп похвалил Сталина Президент США Д...
3    Формальным основанием импичмента Трампа станут...
4    Иранские моряки ошибке подавили протесты Гонко...
Name: newstext, dtype: object

In [143]:
# remove rare words
all_words = " ".join( data.newstext ).split()
rare_words = pd.Series( all_words ).value_counts()[ -2000 : ]
rare_words.sort_values

<bound method Series.sort_values of Дзержинский.       1
голубей.           1
препятствий.       1
корпоративную      1
берется            1
«Прошу             1
вечеринках         1
Ада                1
женщина»           1
американке         1
непризнанную       1
портативный        1
раз:               1
полночь,           1
критикуя           1
«Дон»,             1
прихоти.           1
Нидерландов:       1
чину,              1
4.                 1
язычковый          1
черепами           1
дому,              1
недостойного       1
убедился           1
спёр               1
плиткой,           1
идеи»,             1
гегемонией         1
Воробью            1
                  ..
перемены           1
Здесь,             1
«накрыли»          1
дзёсэя.            1
сыты,              1
огораживания       1
столица,           1
поребрикам.        1
56-летний          1
заблуждение»,      1
привязку           1
Ялте.              1
планшетов,         1
нейтрализовал      1
сегрегацию         

In [144]:
# remove rare_words
data.newstext = data.newstext.apply( lambda t: " ".join( word for word in t.split() if word not in rare_words))
data.newstext.head(5)

0    Россиянам запретят выкладывать интернет неприг...
1    Американский генерал заявил готовности стрелят...
2    Дональд Трамп похвалил Сталина Президент США Д...
3    Формальным основанием импичмента Трампа станут...
4    Иранские моряки ошибке подавили протесты Гонко...
Name: newstext, dtype: object

In [145]:
# remove numbers
data.newstext = data.newstext.apply(lambda t: " ".join(word for word in t.split() if not word.isnumeric() ) )

In [147]:
!pip install stanfordnlp

    100% |████████████████████████████████| 163kB 933kB/s ta 0:00:01
    100% |████████████████████████████████| 81.1MB 220kB/s ta 0:00:011 5% |█▉                              | 4.8MB 2.0MB/s eta 0:00:38    11% |███▋                            | 9.1MB 3.0MB/s eta 0:00:25    54% |█████████████████▍              | 43.9MB 4.8MB/s eta 0:00:08    62% |███████████████████▉            | 50.3MB 5.3MB/s eta 0:00:06    63% |████████████████████▍           | 51.7MB 4.5MB/s eta 0:00:07�███         | 58.3MB 840kB/s eta 0:00:28    78% |█████████████████████████       | 63.3MB 22.9MB/s eta 0:00:01    78% |█████████████████████████▎      | 64.1MB 7.0MB/s eta 0:00:03    89% |████████████████████████████▋   | 72.4MB 5.3MB/s eta 0:00:02


In [149]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/viktoriakoroleva/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [150]:
data['newstext'] = data['newstext'].apply(preprocess_text) 
data.head(5)

,id,url,author,date,tags,type_of_article,newstext,uppercase_count,word_count,avg_word_len
0,0,https://panorama.pub/28367-rossiyanam-zapretya...,Виталий Манн,2019-12-19,Политика,fake,россиянин запрещать выкладывать интернет непри...,7,215,7.060465
1,1,https://panorama.pub/28365-amerikanskij-genera...,Бронислав Рубенштейн,2019-12-19,Политика,fake,американский генерал заявлять готовность стрел...,5,181,5.911602
2,2,https://panorama.pub/28356-donald-tramp-pohval...,Виталий Манн,2019-12-19,Политика,fake,дональд трамп похвалить сталин президент сша д...,5,163,5.889571
3,3,https://panorama.pub/27498-formalnym-osnovanie...,Бронислав Рубенштейн,2019-12-18,Политика,fake,формальный основание импичмент трамп становить...,1,183,6.573770
4,4,https://panorama.pub/28237-iran-oshibochno-pod...,Диана Арутюнян,2019-12-18,Политика,fake,иранский моряк ошибка подавлять протест гонкон...,4,231,6.346320


In [151]:
data.author.value_counts()

None                                      2052
Виталий Манн                               954
Борис Гонтермахер                          470
Бронислав Рубенштейн                       361
Максим Фрейденберг                          78
Гедиминас Кагановичюс                       64
Изяслав Тарашкевич                          59
Григорий Касаткин                           58
Соломон Жидков                              51
Марк Хлестаков                              50
Гермослав Томилин                           44
Саймон Пирч                                 36
Антон Мицкелюнас                            36
Эммануил Щекочихин-Крестовоздвиженский      33
Илья Васильеостровский                      28
Вагиф Джавадов                              24
Андрей Ренье                                23
Егор Гайдай                                 21
Егор Крымов                                 20
Константин Львович                          19
Алексей Фарисеев                            18
Ян Цмок      

In [152]:
data.tags.value_counts()

Общество                                                                                                                                                                                   1722
Политика                                                                                                                                                                                    637
Большая пресс-конференция Владимира Путина, Владимир Путин, Россия, Политика                                                                                                                180
Михаил Горбачев, Общество, Политика                                                                                                                                                         130
Общество Политика                                                                                                                                                                           110
Росприроднадзор, Правительство РФ, Влади

In [154]:
# Convert our target -- label (categorical) into numeric format
data['type_of_article'] = data['type_of_article'].factorize()[0]
data['type_of_article'].value_counts()

# fake 2870
# real 2052

0    2870
1    2052
Name: type_of_article, dtype: int64

In [159]:
# tfidf for feature extraction from text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

# import ML models 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import LinearSVC, SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingClassifier

#  Metrices for model evalaution 
from sklearn import metrics
from sklearn.metrics import classification_report , accuracy_score, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_score


In [160]:
tfidf = TfidfVectorizer()
features = tfidf.fit_transform(data.newstext).toarray()
features_data = pd.DataFrame(features)

In [164]:
models = [
    RandomForestClassifier(n_estimators=200, max_depth=3, random_state=0),
    LinearSVC(),
    MultinomialNB(),
    LogisticRegression(random_state=0),
]

# define cross validation
CV = 5
cv_data = pd.DataFrame(index=range(CV * len(models)))
entries = []

for model in models:
    model_name = model.__class__.__name__
    accuracies = cross_val_score(model, features, data.type_of_article, scoring='accuracy', cv=CV)
    
    for fold_idx, accuracy in enumerate(accuracies):
        entries.append((model_name, fold_idx, accuracy))

cv_data = pd.DataFrame(entries, columns=['model_name', 'fold_idx', 'accuracy'])
cv_data.groupby('model_name').accuracy.mean()

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

model_name
LinearSVC                 0.962805
LogisticRegression        0.962805
MultinomialNB             0.964228
RandomForestClassifier    0.886006
Name: accuracy, dtype: float64

In [165]:
cv_data

,model_name,fold_idx,accuracy
0,RandomForestClassifier,0,0.916751
1,RandomForestClassifier,1,0.932995
2,RandomForestClassifier,2,0.895325
3,RandomForestClassifier,3,0.895325
4,RandomForestClassifier,4,0.789634
5,LinearSVC,0,1.000000
6,LinearSVC,1,1.000000
7,LinearSVC,2,1.000000
8,LinearSVC,3,1.000000
9,LinearSVC,4,0.814024


In [167]:
# As two models are performing way better than others:
# LinearSVC -89.7 %, LogisticRegression - 88.6%
# let's dig deeper and see the Precision, Recall, F1 Score
X_train, X_test, y_train, y_test = train_test_split(features, data.type_of_article, test_size = 0.3, random_state = 0)

In [168]:
lr = LogisticRegression()
lr.fit(X_train, y_train)

print("Accuracy of LogisticRegression = ")
print(lr.score(X_test, y_test))
print("\n")

y_pred_lr = lr.predict(X_test)
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred_lr))
print("\n")
print(classification_report(y_test, y_pred_lr))

/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Accuracy of LogisticRegression = 
0.9817197020988491


Confusion Matrix
[[842   0]
 [ 27 608]]


              precision    recall  f1-score   support

           0       0.97      1.00      0.98       842
           1       1.00      0.96      0.98       635

   micro avg       0.98      0.98      0.98      1477
   macro avg       0.98      0.98      0.98      1477
weighted avg       0.98      0.98      0.98      1477



In [172]:
svm = LinearSVC()
svm.fit(X_train, y_train)

print("Accuracy of LinearSVC = ")
print(svm.score( X_test, y_test ))
print("\n")

y_pred_svm = svm.predict(X_test)
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred_svm))
print("\n")
print(classification_report(y_test, y_pred_svm))

Accuracy of LinearSVC = 
1.0


Confusion Matrix
[[842   0]
 [  0 635]]


              precision    recall  f1-score   support

           0       1.00      1.00      1.00       842
           1       1.00      1.00      1.00       635

   micro avg       1.00      1.00      1.00      1477
   macro avg       1.00      1.00      1.00      1477
weighted avg       1.00      1.00      1.00      1477



In [170]:
mnb = MultinomialNB()
mnb.fit(X_train, y_train)

print("Accuracy of MultinomialNB = ")
print(mnb.score(X_test, y_test))
print("\n")

y_pred_mnb = mnb.predict(X_test)
print("Confusion Matrix")
print(confusion_matrix(y_test, y_pred_mnb))
print("\n")
print(classification_report(y_test, y_pred_mnb))

Accuracy of MultinomialNB = 
0.982396750169262


Confusion Matrix
[[842   0]
 [ 26 609]]


              precision    recall  f1-score   support

           0       0.97      1.00      0.98       842
           1       1.00      0.96      0.98       635

   micro avg       0.98      0.98      0.98      1477
   macro avg       0.99      0.98      0.98      1477
weighted avg       0.98      0.98      0.98      1477

